In [119]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
from bokeh.plotting import figure
from bokeh.io import output_notebook, push_notebook, show
from bokeh.models import CustomJS, Slider, HoverTool,ColumnDataSource
from bokeh.layouts import row
from bokeh.charts import Scatter

In [120]:
produceTotal = pd.read_excel("Electricity Generation.xls.xlsx")
produceTotal=produceTotal.set_index('Electricity generation total (kilowatt-hours)')
country_list=produceTotal.index.tolist()
year_list=produceTotal.columns.tolist()
year_list=[str(i) for i in year_list]

In [121]:
consumeTotalRaw = pd.read_excel("Indicator_Electricity consumption total.xlsx")
consumeTotalRaw=consumeTotalRaw.set_index('Electricity consumption, total (kWh)')
consumeTotalRaw=consumeTotalRaw.loc[:,year_list]
consumeTotal=consumeTotalRaw.loc[country_list]

In [123]:
output_notebook()
data = {
    'x_values' : produceTotal[1990],
    'y_values' : consumeTotal['1990'],
    'country' : country_list
}
# source1=ColumnDataSource(produceTotal)
# source2=ColumnDataSource(consumeTotal)
source=ColumnDataSource(data=data)
power = 0.5
hover = HoverTool(tooltips=[
    ("Country", "@country"),
    ("(x,y)", "($x, $y)")
])
fig = figure(plot_width=500, plot_height=500, tools=[hover,'wheel_zoom','box_zoom','box_select','reset'],title="Energy consumption vs. Production",x_axis_type="log",y_axis_type="log")
plt = fig.circle(x='x_values', y='y_values',size=10,color='blue',alpha=0.3,source=source)
fig.xaxis.axis_label= "Total Energy Production in kWh (Log scale)"
fig.yaxis.axis_label= "Total Energy Consumption in kWh (Log scale)"
def update_year(year):
    plt.data_source.data['x_values']=produceTotal[year]
    plt.data_source.data['y_values']=consumeTotal[str(year)]
    push_notebook(handle=bokeh_handle)  

callback = CustomJS(code="""
if (IPython.notebook.kernel !== undefined) {
    var kernel = IPython.notebook.kernel;
    cmd = "update_year(" + cb_obj.value + ")";
    kernel.execute(cmd, {}, {});}
""")

slider = Slider(start=1990, 
                end=2008,
                value=1990,
                step=1,
                title="Year",
                callback=callback)
bokeh_handle = show(row(fig, slider), notebook_handle=True)

Loading BokehJS ...